In [79]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [80]:
tpe_stations = pd.read_csv('TPE_bike_station.csv')
df = pd.read_csv('2023_11_bike_usage_history.csv',index_col=0)

In [81]:
feature_set = tpe_stations['station_no'].to_frame()
feature_set['station'] = tpe_stations['name_tw']

In [82]:
df.columns=['lend_time','lend_station_name','return_time','return_station_name','usage_time','source_date']
df['source_date'] = pd.to_datetime(df['source_date'])
df['lend_date'] = pd.to_datetime(df['lend_time']).dt.date
df['lend_hour'] = pd.to_datetime(df['lend_time']).dt.hour
df['return_date'] = pd.to_datetime(df['return_time']).dt.date
df['return_hour'] = pd.to_datetime(df['return_time']).dt.hour
df = df.drop(['lend_time','return_time'],axis=1)
df['usage_time'] = pd.to_timedelta(df['usage_time']).dt.total_seconds().astype('int')

In [83]:
# average daily use (weekday)
# average daily use (weekend)


In [84]:
df['day_of_week'] = pd.to_datetime(df['source_date']).dt.weekday

In [85]:
weekday = df[~df['day_of_week'].isin([5,6])]
weekday = pd.concat([weekday['lend_station_name'],weekday['return_station_name']]).rename('station').value_counts().to_frame().reset_index()
weekday = weekday[weekday['station'].isin(tpe_stations['name_tw'])]

In [86]:
weekend = df[~df['day_of_week'].isin(range(0,5))]
weekend = pd.concat([weekend['lend_station_name'],weekend['return_station_name']]).rename('station').value_counts().to_frame().reset_index()
weekend = weekend[weekend['station'].isin(tpe_stations['name_tw'])]

In [87]:
weekday['count'] = weekday['count']/df['source_date'].nunique()
weekend['count'] = weekend['count']/df['source_date'].nunique()


In [88]:
avg_use = pd.merge(weekday,weekend,how='outer',on='station',suffixes=['_weekday','_weekend'])
avg_use = avg_use.fillna(0)
del weekday,weekend

In [89]:
feature_set = feature_set.merge(avg_use,how='inner',on='station')
feature_set.shape
del avg_use

In [90]:
# Peak Usage Times top 5 (weekday)
# Peak Usage Times top 5 (weekend)

In [91]:
weekday = df[~df['day_of_week'].isin([5,6])]

In [92]:
col_name = ['station','hour']
weekday = pd.concat([
    weekday[['lend_station_name','lend_hour']].set_axis(col_name,axis=1),
    weekday[['return_station_name','return_hour']].set_axis(col_name,axis=1),
])

In [93]:
weekday_traffic_counts = weekday.groupby(['station', 'hour']).value_counts().reset_index(name='counts')


In [94]:
weekday_traffic_counts = weekday_traffic_counts.groupby('station', group_keys=False)\
    .apply(lambda x: x.sort_values(by='counts', ascending=False).head(5))

In [95]:
weekday_traffic_counts['rank'] = weekday_traffic_counts.groupby('station')['counts'].rank(method='first', ascending=False)

# Pivot the DataFrame to wide format
weekday_pivot_df = weekday_traffic_counts.pivot(index='station', columns='rank', values=['hour', 'counts'])

In [96]:
weekday_pivot_df.columns = [f'{col[0]}_top{int(col[1])}' for col in weekday_pivot_df.columns]
weekday_pivot_df.reset_index(inplace=True)

In [97]:
weekend = df[~df['day_of_week'].isin(range(0,5))]
col_name = ['station','hour']
weekend = pd.concat([
    weekend[['lend_station_name','lend_hour']].set_axis(col_name,axis=1),
    weekend[['return_station_name','return_hour']].set_axis(col_name,axis=1),
])
weekend_traffic_counts = weekend.groupby(['station', 'hour']).value_counts().reset_index(name='counts')
weekend_traffic_counts = weekend_traffic_counts.groupby('station', group_keys=False)\
    .apply(lambda x: x.sort_values(by='counts', ascending=False).head(5))
weekend_traffic_counts['rank'] = weekend_traffic_counts.groupby('station')['counts'].rank(method='first', ascending=False)

# Pivot the DataFrame to wide format
weekend_pivot_df = weekend_traffic_counts.pivot(index='station', columns='rank', values=['hour', 'counts'])
weekend_pivot_df.columns = [f'{col[0]}_top{int(col[1])}' for col in weekend_pivot_df.columns]
weekend_pivot_df.reset_index(inplace=True)

In [98]:
pivot_data = pd.merge(weekday_pivot_df,weekend_pivot_df,how='outer',on='station',suffixes=['_weekday','_weekend'])
del weekday,weekend,weekday_pivot_df,weekend_pivot_df

In [99]:
pivot_data = pivot_data[pivot_data['station'].isin(tpe_stations['name_tw'])]

In [100]:
feature_set = feature_set.merge(pivot_data,how='inner',on='station')
feature_set.shape
del pivot_data

In [101]:
# Usage Variability (daily) 變異係數
# Usage Variability (hourly) 變異係數
# Usage Variability (weekend/weekday) 變異係數
col_name = ['station','hour','source_date']
all_access = pd.concat([
    df[['lend_station_name','lend_hour','source_date']].set_axis(col_name,axis=1),
    df[['return_station_name','return_hour','source_date']].set_axis(col_name,axis=1),
])

In [102]:
# groupby_hour
groupby_hour = all_access.groupby(['station','hour']).size().reset_index(name='traffic_count')
groupby_hour = groupby_hour.pivot(index='station',columns='hour',values='traffic_count').fillna(0).astype('int')
groupby_hour['cv'] = np.std(groupby_hour.to_numpy(),axis=1)/np.mean(groupby_hour.to_numpy(),axis=1)
hourly_cv = groupby_hour.reset_index()[['station','cv']]
hourly_cv.columns.name=None
del groupby_hour
hourly_cv.head()


,station,cv
0,3樓客服中心,2.828427
1,?公公園,0.674676
2,?寮公園,4.795832
3,一壽橋,0.761308
4,一江公園,0.768149


In [103]:
groupby_date = all_access.groupby(['station','source_date']).size().reset_index(name='traffic_count')
groupby_date = groupby_date.pivot(index='station',columns='source_date',values='traffic_count').fillna(0).astype('int')
groupby_date['cv'] = np.std(groupby_date.to_numpy(),axis=1)/np.mean(groupby_date.to_numpy(),axis=1)
daily_cv = groupby_date.reset_index()[['station','cv']]
daily_cv.columns.name=None
del groupby_date
daily_cv.head()


,station,cv
0,3樓客服中心,3.201562
1,?公公園,0.203533
2,?寮公園,5.385165
3,一壽橋,0.312968
4,一江公園,0.257237


In [104]:
all_access['day_of_week'] = pd.to_datetime(all_access['source_date']).dt.weekday
all_access['weekday_weekend'] = np.where(all_access['day_of_week'].isin(range(0,5)),'weekday','weekend')

In [105]:
groupby_weekday_weekend = all_access.groupby(['station','weekday_weekend']).size().reset_index(name='traffic_count')
groupby_weekday_weekend = groupby_weekday_weekend.pivot(index='station',columns='weekday_weekend',values='traffic_count').fillna(0).astype('int')
groupby_weekday_weekend['cv'] = np.std(groupby_weekday_weekend.to_numpy(),axis=1)/np.mean(groupby_weekday_weekend.to_numpy(),axis=1)
weekday_weekend_cv = groupby_weekday_weekend.reset_index()[['station','cv']]
weekday_weekend_cv.columns.name=None
del groupby_weekday_weekend
weekday_weekend_cv.head()


,station,cv
0,3樓客服中心,0.500000
1,?公公園,0.433641
2,?寮公園,1.000000
3,一壽橋,0.528730
4,一江公園,0.648204


In [106]:
del all_access

In [107]:
feature_set = feature_set.merge(hourly_cv,how='inner',on='station')
feature_set = feature_set.merge(daily_cv,how='inner',on='station')
feature_set = feature_set.merge(weekday_weekend_cv,how='inner',on='station')
feature_set.shape
del hourly_cv,daily_cv,weekday_weekend_cv

In [108]:
# Duration of Use (weekday)
# Duration of Use (weekend)
weekday_duration = df[~df['day_of_week'].isin([5,6])].groupby('lend_station_name')['usage_time'].mean().reset_index()
weekend_duration = df[~df['day_of_week'].isin(range(0,5))].groupby('lend_station_name')['usage_time'].mean().reset_index()
duration_data = pd.merge(weekday_duration,weekend_duration,how='outer',on='lend_station_name',suffixes=['_weekday','_weekend'])
duration_data = duration_data.rename({'lend_station_name':'station'},axis=1)
duration_data.head()

,station,usage_time_weekday,usage_time_weekend
0,3樓客服中心,478.333333,589.000000
1,?公公園,1162.664787,1189.630844
2,一壽橋,1729.235751,2475.450820
3,一江公園,1001.648606,1125.544118
4,三張犁,1000.225486,1102.718121


In [109]:
feature_set = feature_set.merge(duration_data,how='inner',on='station')
feature_set.shape
del duration_data

In [110]:
# percentage of top3 linking stations
# self returning percentage

link_count = df.groupby('lend_station_name')['return_station_name'].value_counts(normalize=True).reset_index(name='proportion')
top3_data = link_count.groupby('lend_station_name',group_keys=False).apply(lambda x: x.sort_values(by='proportion', ascending=False).head(3)['proportion'].sum()).reset_index(name='top3_sum_percentage')
top3_data.rename({'lend_station_name':'station'},
                            axis=1,inplace=True)
top3_data.head()

,station,top3_sum_percentage
0,3樓客服中心,1.000000
1,?公公園,0.083735
2,一壽橋,0.326772
3,一江公園,0.136215
4,三張犁,0.268297


In [111]:
self_returning_data = link_count[link_count['lend_station_name']==link_count['return_station_name']]
self_returning_data = self_returning_data[['lend_station_name','proportion']]
self_returning_data.rename({'lend_station_name':'station',
                            'proportion':'self_returning_proportion'},
                            axis=1,inplace=True)
self_returning_data.head()

,station,self_returning_proportion
0,3樓客服中心,1.000000
1,?公公園,0.044378
558,一壽橋,0.256890
675,一江公園,0.072692
936,三張犁,0.105175


In [112]:
del link_count

In [113]:
feature_set = feature_set.merge(top3_data,how='inner',on='station')
feature_set = feature_set.merge(self_returning_data,how='inner',on='station')
feature_set.shape
del top3_data, self_returning_data

In [114]:
# Usage Variability (Seasonality) 目前只有一個月沒辦法有這個資料


In [115]:
#Turnover Rate (不知道怎麼算？
traffic_count = df.groupby(by=['lend_station_name']).size().reset_index(name='counts')
traffic_count['daily_count'] = traffic_count['counts']/df['source_date'].nunique()

In [116]:
traffic_count = tpe_stations[['name_tw','parking_spaces']].merge(traffic_count,how='inner',left_on='name_tw',right_on='lend_station_name')

In [117]:
traffic_count['daily_turnover_rate'] = traffic_count['daily_count']/traffic_count['parking_spaces']
traffic_count = traffic_count[['name_tw','daily_turnover_rate']]
traffic_count.rename({'name_tw':'station'},axis=1,inplace=True)
traffic_count.head()

,station,daily_turnover_rate
0,捷運科技大樓站,18.127381
1,復興南路二段273號前,5.868254
2,國北教大實小東側門,4.352083
3,和平公園東側,8.660606
4,辛亥復興路口西北側,8.900000


In [118]:
feature_set = feature_set.merge(traffic_count,how='inner',on='station')
feature_set.shape
del traffic_count

### next nearest station distance

In [119]:
# next nearest station distance
import geopy.distance

In [120]:
def find_closest_distance(row,base_point):
    other_point = row[['lat','lng']].to_numpy()
    return geopy.distance.geodesic(base_point,other_point).m
    

In [121]:
#time complexity O(n**2)
# distance = []
# for idx,row in tqdm(tpe_stations.iterrows()):
#     point_A = row[['lat','lng']].to_numpy()
#     distance_list = tpe_stations.apply(find_closest_distance,base_point=point_A,axis=1)
#     closest_distance = distance_list.sort_values().values[1]
#     distance.append(closest_distance)

In [122]:
#time complexity O(n**2) but it's n(n-1)/2
coords = tpe_stations[['lat', 'lng']].to_numpy()
    
# Initialize a matrix to store distances
dist_matrix = np.zeros((len(coords), len(coords)))

# Compute geodesic distance between each pair of points
for i in tqdm(range(len(coords))):
    for j in range(i + 1, len(coords)):
        dist = geopy.distance.geodesic(coords[i], coords[j]).m
        dist_matrix[i, j] = dist
        dist_matrix[j, i] = dist  # since distance is symmetric

# Find the minimum distance for each point, ignoring the zero (self-distance)
# closest_distances = np.min(np.where(dist_matrix > 0, dist_matrix, np.inf), axis=1)
# return closest_distances

100%|██████████| 1413/1413 [00:48<00:00, 29.19it/s] 


In [123]:
closest_distances = np.min(np.where(dist_matrix > 0, dist_matrix, np.inf), axis=1)

In [124]:
tpe_stations['closest_distances'] = closest_distances

In [125]:
# number of station in radius
tpe_stations['R500_station_count'] = np.sum(dist_matrix<500,axis=0)-1

In [126]:
tpe_stations.columns

Index(['station_no', 'name_tw', 'district_tw', 'address_tw', 'parking_spaces',
       'lat', 'lng', 'city_code', 'closest_distances', 'R500_station_count'],
      dtype='object')

In [127]:
feature_set = feature_set.merge(tpe_stations[['name_tw','closest_distances', 'R500_station_count']],how='inner',
                                left_on='station',
                                right_on='name_tw')
feature_set.shape

(1292, 35)

In [128]:
del df

### 用即時資料萃取

In [129]:
ubike_rt = pd.read_csv('ubike_0501.csv',index_col=0)
ubike_rt.drop_duplicates(inplace=True)

In [130]:
# 缺車/正常/缺位比例 （缺車風險）還沒看風險怎麼算 先以即時資料的比例來看
# 0 / 10% or 5 / normal
ubike_rt['thres'] = np.min(np.column_stack(((ubike_rt['tot']*0.1).to_numpy(),np.full(len(ubike_rt),5))),axis=1)

In [131]:
def bike_status(row):
    space = row['bemp']
    bike = row['sbi']
    thres = row['thres']
    if space == 0:
        return '滿車'
    elif bike ==0:
        return '沒車'
    elif space <thres:
        return '快滿車'
    elif bike <thres:
        return '快沒車'
    else:
        return '正常'

In [132]:
ubike_rt['station_status'] = ubike_rt.apply(bike_status,axis=1)

In [133]:
ubike_rt['station'] = ubike_rt['sna'].apply(lambda x: x.split('_')[1])

In [134]:
status_percentage = ubike_rt[['station','station_status']].groupby('station').value_counts(normalize=True).reset_index(name='percentage')

In [135]:
status_percentage = status_percentage.pivot(index='station',columns='station_status',values='percentage').reset_index()
status_percentage.fillna(0,inplace=True)


In [136]:
status_percentage['empty_risk_percentage'] = status_percentage['快沒車']+status_percentage['沒車']
status_percentage['full_risk_percentage'] = status_percentage['快滿車']+status_percentage['滿車']
status_percentage.head()

station_status,station,快沒車,快滿車,正常,沒車,滿車,empty_risk_percentage,full_risk_percentage
0,一壽橋,0.018837,0.031755,0.932185,0.013455,0.003767,0.032293,0.035522
1,一江公園,0.247578,0.000000,0.604413,0.148009,0.000000,0.395587,0.000000
2,三張犁,0.144241,0.022605,0.768030,0.055436,0.009688,0.199677,0.032293
3,三民公園,0.051130,0.041442,0.872982,0.020990,0.013455,0.072121,0.054898
4,三民公園(塔悠路),0.050592,0.003767,0.900431,0.035522,0.009688,0.086114,0.013455


In [137]:
feature_set = feature_set.merge(status_percentage[['station','empty_risk_percentage', 'full_risk_percentage']],how='inner',
                                on='station')
feature_set.shape
del status_percentage

In [138]:
# delta value mean
ubike_rt['delta'] = np.abs(ubike_rt.groupby('sna')['sbi'].diff())

In [139]:
ubike_rt = ubike_rt.dropna()

In [140]:
avg_delta_by_station = ubike_rt.groupby('station')['delta'].mean().reset_index()

In [141]:
del ubike_rt

In [142]:
feature_set = feature_set.merge(avg_delta_by_station,how='inner',on='station')
del avg_delta_by_station
feature_set.shape

(1276, 38)

In [143]:
feature_set.drop('name_tw',axis=1,inplace=True)

In [144]:
feature_set.to_csv('bike_station_grouping_features.csv',index=False)